In [173]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [174]:
PG_HOST='localhost'
PG_DATABASE='pi3'
PG_USER='arturo'
PG_PASSWORD='vaifunfar123'

STRING_CONEXAO = 'host='+ PG_HOST + ' port=5432' + ' dbname=' + \
              PG_DATABASE + ' user=' + PG_USER + ' password=' + PG_PASSWORD

LOCAL_DADOS = 'dados/'
ARQUIVO_BASE_PRINCIPAL = 'music_data.csv'
ARQUIVO_GENERO = 'music_features.csv'
ARQUIVO_BASE_TOP_200 = 'top_200.csv'
ARQUIVO_BASE_TOP_200_BRAZIL = 'top_200_brazil.csv'
ARQUIVO_BASE_TOP_200_BRAZIL_ARGENTINA = 'top_200_brazil_argentina.csv'
# ARQUIVO_BASE_TOP_100_20_ANOS = ['Top2001.csv','Top2002.csv','Top2003.csv','Top2004.csv','Top2005.csv',
#                                 'Top2006.csv','Top2007.csv','Top2008.csv','Top2009.csv','Top2010.csv',
#                                 'Top2011.csv','Top2012.csv','Top2013.csv','Top2014.csv','Top2015.csv',
#                                 'Top2016.csv','Top2017.csv','Top2018.csv','Top2019.csv']

COLUNAS_BASE_PRINCIPAL = ['valence','year','acousticness','artists','danceability','duration_ms',
                            'energy','explicit','id','instrumentalness','key','liveness','loudness',
                            'mode','name','popularity','speechiness','tempo']

COLUNAS_BASE_TOP_200 = ['URL', 'Streams', 'Position', 'Date', 'name']
COLUNAS_BASE_TOP_100_20_ANOS = ['id', 'year']
COLUNAS_GENERO = ['track_id', 'genre']

DDL_OBJETOS = (
    'DROP TABLE IF EXISTS spotify',
    'DROP TABLE IF EXISTS artists',
    'DROP TABLE IF EXISTS genre',
    'DROP TABLE IF EXISTS year',
    'DROP TABLE IF EXISTS country',
    'DROP TABLE IF EXISTS songs',
    
    'DROP INDEX IF EXISTS idx_genre_track_id',    
    
    'CREATE TABLE IF NOT EXISTS artists (' 
        'artists_id int,' 
        'artists varchar,' 
        'CONSTRAINT PK_artists_id PRIMARY KEY(artists_id)' 
    ')',
    
    'CREATE TABLE IF NOT EXISTS genre (' 
        'genre_id int,' 
        'genre varchar,' 
        'track_id varchar,' 
        'CONSTRAINT PK_genre_id PRIMARY KEY(genre_id)' 
    ')',
    
    'CREATE TABLE IF NOT EXISTS year (' 
        'year int,'
        'CONSTRAINT PK_year PRIMARY KEY(year)' 
    ')',
    
    'CREATE TABLE IF NOT EXISTS country ('         
        'country varchar,'        
        'CONSTRAINT PK_country_id PRIMARY KEY(country)' 
    ')',
    
    'CREATE TABLE IF NOT EXISTS songs (' 
        'track_id varchar,' 
        'song varchar,'
        'CONSTRAINT PK_track_id PRIMARY KEY(track_id)'
    ')',
    
    'CREATE TABLE IF NOT EXISTS spotify (' 
        'track_id varchar,'                 
        'popularity int,' 
        'year int,'                     
        'artists_id int,'        
        'valence float,' 
        'acousticness float,' 
        'danceability float,' 
        'duration_ms int,' 
        'energy float,' 
        'explicit int,' 
        'instrumentalness float,' 
        'key int,' 
        'liveness float,' 
        'loudness float,' 
        'mode int,' 
        'speechiness float,' 
        'tempo float,' 
    
        'streams_date date,'
        'streams_year int,'
        'streams int,'
        'position int,'
        'country varchar,'
    
        'features_array numeric[]'            
    ')',
    
    'CREATE INDEX idx_genre_track_id ON genre(track_id)',     
)

In [175]:
import psycopg2
import strings as strlib

def criar_objetos_banco(ddl):

    cursor = None
    conexao = None

    try:
        conexao = psycopg2.connect(strlib.STRING_CONEXAO)
        cursor = conexao.cursor()

        for instrucao in ddl:
            cursor.execute(instrucao)

        cursor.close()
        conexao.commit()

        print('OBJETOS CRIADOS COM SUCESSO.')

    except (Exception, psycopg2.DatabaseError) as error:
#        conexao.rollback()
        print(error)
    finally:
        if conexao is not None:
            conexao.close()

#--------------------------------------------------

def inserir_dados(dataframe, tabela, campos):
    
    conexao = None
    cursor = None
    tupla = None
    
    try:

        conexao = psycopg2.connect(strlib.STRING_CONEXAO)
        cursor = conexao.cursor()   

        print('[' + tabela.upper() + '] INSERINDO DADOS NA TABELA...')

        #Povoa a tabela de aferições
        
        if (len(dataframe) == 1):
            for i, linha in dataframe.iterrows():
                sql = 'INSERT INTO ' + tabela + '(' + campos + ') ' \
                    'VALUES (' + '%s)'
                tupla = tuple(linha)
                cursor.execute(sql, tuple(linha))
        else:
            for i, linha in dataframe.iterrows():
                sql = 'INSERT INTO ' + tabela + '(' + campos + ') ' \
                    'VALUES (' + '%s,' *(len(linha)-1) + '%s)'
                tupla = tuple(linha)
                cursor.execute(sql, tuple(linha))

        conexao.commit()

        print('[' + tabela.upper() + '] REGISTROS INSERIDOS COM SUCESSO.')

    except (Exception, psycopg2.DatabaseError) as error:
#        conexao.rollback()
        print(error)
        print(tupla)
    finally:
        if conexao is not None:
            conexao.close()

In [176]:
import psycopg2
import pandas as pd
import spotifylib as slib
import strings as strlib

In [177]:
#Bloco para criação dos objetos de banco
slib.criar_objetos_banco(strlib.DDL_OBJETOS)

OBJETOS CRIADOS COM SUCESSO.


In [178]:
df_base_principal = pd.read_csv(strlib.LOCAL_DADOS + strlib.ARQUIVO_BASE_PRINCIPAL, encoding='utf-8')[strlib.COLUNAS_BASE_PRINCIPAL]
df_base_principal = df_base_principal.rename(columns={'id': 'track_id'})
df_base_principal = df_base_principal.rename(columns={'name': 'song'})
df_base_principal = df_base_principal[df_base_principal.year.isin([2018,2019,2020])]
#df_base_principal

In [179]:
#monta o dataframe para a tabela de songs
df_songs = df_base_principal[['track_id','song']]

df_base_principal = df_base_principal.drop(['song'],axis=1)

#inserir dados na tabela de artistas
TABELA = 'songs'
CAMPOS = 'track_id, song'

slib.inserir_dados(df_songs, TABELA, CAMPOS)

[SONGS] INSERINDO DADOS NA TABELA...
[SONGS] REGISTROS INSERIDOS COM SUCESSO.


In [180]:
#monta o dataframe para a tabela de artistas
artistas = list(df_base_principal['artists'])
df_artists = pd.DataFrame(artistas, columns=['artists'])
id_artists_column = list(range(1,len(df_artists)+1))

df_artists['id_artists'] = id_artists_column

df_base_principal['id_artists'] = id_artists_column
df_base_principal = df_base_principal.drop(['artists'],axis=1)

#inserir dados na tabela de artistas
TABELA = 'artists'
CAMPOS = 'artists, artists_id'

slib.inserir_dados(df_artists, TABELA, CAMPOS)

[ARTISTS] INSERINDO DADOS NA TABELA...
[ARTISTS] REGISTROS INSERIDOS COM SUCESSO.


In [181]:
#monta o dataframe para a tabela de anos
anos = list(df_base_principal['year'])
anos = list(dict.fromkeys(anos)) #remove duplicados
df_anos = pd.DataFrame(anos)

#inserir dados na tabela de anos
TABELA = 'year'
CAMPOS = 'year'

slib.inserir_dados(df_anos, TABELA, CAMPOS)

[YEAR] INSERINDO DADOS NA TABELA...
[YEAR] REGISTROS INSERIDOS COM SUCESSO.


In [182]:
df_anos

,0
0,2018
1,2019
2,2020


In [183]:
#trabalho para montar gênero musical
df_genero = pd.read_csv(strlib.LOCAL_DADOS + strlib.ARQUIVO_GENERO, encoding='utf-8')[strlib.COLUNAS_GENERO]
genre_column_id = range(1,len(df_genero['genre'])+1)
df_genero['genre_id'] = genre_column_id

#inserir dados na tabela de anos
TABELA = 'genre'
CAMPOS = 'track_id, genre, genre_id'

slib.inserir_dados(df_genero, TABELA, CAMPOS)

[GENRE] INSERINDO DADOS NA TABELA...
[GENRE] REGISTROS INSERIDOS COM SUCESSO.


In [184]:
CAMPOS

'track_id, genre, genre_id'

In [185]:
TABELA

'genre'

In [186]:
#trabalho para montar informações de países
#aqui, há o top 200 em vários países nos anos de 2017 a 2021

#---------------------------------------
#Esta seção comentada tratava o arquivo inteiro de 1.2GB e 9 milhões de linhas
# df_country = pd.read_csv(strlib.LOCAL_DADOS + strlib.ARQUIVO_BASE_TOP_200, encoding='utf-8')[strlib.COLUNAS_BASE_TOP_200]
# # df_country = df_country[df_country.name == 'Brazil']
# df_country = df_country[df_country.name.isin(['Brazil','Argentina']) && (df_country.year >= 2018 && df_country.year <= 2020)]
# df_country = df_country.rename(columns={'URL': 'url', 'name': 'country', 'Streams': 'streams', 'Position': 'position', 'Date': 'date'})

# country_column_id = range(1,len(df_country['country'])+1)
# df_country['country_id'] = country_column_id

# df_country['track_id'] = df_country['url'].str[31:]
# df_country = df_country.drop(['url'],axis=1)

# df_country['year'] = df_country['date'].str[:4]

# df_country.to_csv('dados/top_200_brazil_argentina.csv')

#---------------------------------------
#Aqui, o trabalho será feito apenas no arquivo do brasil e argentina para fins
#de simplicidade. Um arquivo novo foi gerado para tal.
#Utilizamos o comando .to_csv(nome.csv) para geração
df_country = pd.read_csv(strlib.LOCAL_DADOS + strlib.ARQUIVO_BASE_TOP_200_BRAZIL, encoding='utf-8')
countries = list(dict.fromkeys(df_country['country'])) #remove duplicados
df_country_banco = pd.DataFrame(countries, columns=['country'])
df_country_banco

# #inserir dados na tabela de anos
TABELA = 'country'
# CAMPOS = 'streams, position, date, country, country_id, track_id, year'
CAMPOS = 'country'

slib.inserir_dados(df_country_banco, TABELA, CAMPOS)

[COUNTRY] INSERINDO DADOS NA TABELA...
[COUNTRY] REGISTROS INSERIDOS COM SUCESSO.


In [187]:
df_country

,streams,position,date,country,country_id,track_id,year
0,590610,1,07/01/2017,Brazil,1,0EPxmvsG1BY5td4aTOkWBF,2017
1,322448,2,07/01/2017,Brazil,2,1a5Yu5L18qNxVhXx38njON,2017
2,302643,3,07/01/2017,Brazil,3,7qiZfU4dY1lWllzX7mPBI3,2017
3,228938,4,07/01/2017,Brazil,4,45wqPwIH4A5cr2IyyD4WKf,2017
4,221314,5,07/01/2017,Brazil,5,257tme6AhneIKAqVF1HTH4,2017
...,...,...,...,...,...,...,...
180395,95995,196,16/04/2021,Brazil,180396,3MkdA6vwF0ifRl86yzTlJW,2021
180396,95655,197,16/04/2021,Brazil,180397,7sCdzif8vD8XN1jgdYFL1q,2021
180397,95022,198,16/04/2021,Brazil,180398,3oqC4Ta40ft6NnTJwcUTdi,2021
180398,94868,199,16/04/2021,Brazil,180399,63400Pm7x5Awe4NH82oRYe,2021


In [188]:
#trabalho para inserir dados na tabela de fatos
df_spotify = pd.DataFrame([], columns=df_base_principal.columns)
df_spotify = df_spotify.assign(streams_date=[])
df_spotify = df_spotify.assign(streams_year=[])
df_spotify = df_spotify.assign(streams=[])
df_spotify = df_spotify.assign(position=[])
df_spotify = df_spotify.assign(country=[])
###
##df_spotify = df_spotify.assign(features_array=[])

for row_bp in df_base_principal.iterrows():
    track_id = row_bp[1]['track_id']
    df_country_temp = df_country[df_country.track_id == track_id]
    
    for row_country_temp in df_country_temp.iterrows():                        
        df_spotify = df_spotify.append(pd.Series(
            [row_bp[1]['valence'],row_bp[1]['year'],
            row_bp[1]['acousticness'], row_bp[1]['danceability'], row_bp[1]['duration_ms'], row_bp[1]['energy'], row_bp[1]['explicit'],
            row_bp[1]['track_id'], row_bp[1]['instrumentalness'], row_bp[1]['key'], row_bp[1]['liveness'], row_bp[1]['loudness'],
            row_bp[1]['mode'], row_bp[1]['popularity'], row_bp[1]['speechiness'], row_bp[1]['tempo'], row_bp[1]['id_artists'],

            row_country_temp[1]['date'], row_country_temp[1]['year'], row_country_temp[1]['streams'], 
            row_country_temp[1]['position'], row_country_temp[1]['country']],
            
            index = df_spotify. columns
        ),ignore_index=True)            

In [189]:
df_spotify

,valence,year,acousticness,danceability,duration_ms,energy,explicit,track_id,instrumentalness,key,...,mode,popularity,speechiness,tempo,id_artists,streams_date,streams_year,streams,position,country
0,0.218,2018,0.349,0.511,239836,0.566,1,285pBltuF7vW8TeWk8hdRR,0.000000,6,...,0,89,0.2000,83.903,2,09/12/2019,2019.0,96687.0,155.0,Brazil
1,0.218,2018,0.349,0.511,239836,0.566,1,285pBltuF7vW8TeWk8hdRR,0.000000,6,...,0,89,0.2000,83.903,2,10/12/2019,2019.0,88501.0,183.0,Brazil
2,0.120,2018,0.934,0.351,200186,0.296,0,0u2P5u6lvoDfwTYjAADbn4,0.000000,4,...,0,89,0.0333,115.284,5,24/05/2018,2018.0,52373.0,198.0,Brazil
3,0.120,2018,0.934,0.351,200186,0.296,0,0u2P5u6lvoDfwTYjAADbn4,0.000000,4,...,0,89,0.0333,115.284,5,28/05/2018,2018.0,53482.0,175.0,Brazil
4,0.120,2018,0.934,0.351,200186,0.296,0,0u2P5u6lvoDfwTYjAADbn4,0.000000,4,...,0,89,0.0333,115.284,5,29/05/2018,2018.0,60384.0,152.0,Brazil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14649,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,0,72,0.0664,89.717,6046,30/03/2020,2020.0,117456.0,87.0,Brazil
14650,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,0,72,0.0664,89.717,6046,31/03/2020,2020.0,99292.0,120.0,Brazil
14651,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,0,72,0.0664,89.717,6046,01/04/2020,2020.0,86341.0,145.0,Brazil
14652,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,0,72,0.0664,89.717,6046,02/04/2020,2020.0,85979.0,163.0,Brazil


In [224]:
#trabalho para inserir dados na tabela de fatos
# df_base_principal = df_base_principal.drop(['release_date'],axis=1)
# df_base_principal

#insere dados na tabela spotify
TABELA = 'spotify'
CAMPOS = 'valence, year, acousticness, danceability, duration_ms, energy, explicit, track_id, instrumentalness, key, liveness, loudness, mode, popularity, speechiness, tempo, artists_id,streams_date,streams_year,streams,position,country,features_array'

slib.inserir_dados(df_spotify, TABELA, CAMPOS)

[SPOTIFY] INSERINDO DADOS NA TABELA...
[SPOTIFY] REGISTROS INSERIDOS COM SUCESSO.


In [151]:
df_spotify.columns

Index(['valence', 'year', 'acousticness', 'danceability', 'duration_ms',
       'energy', 'explicit', 'track_id', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'popularity', 'speechiness', 'tempo', 'id_artists',
       'streams_date', 'streams_year', 'streams', 'position', 'country',
       'features_array'],
      dtype='object')

In [194]:
df_spotify

,valence,year,acousticness,danceability,duration_ms,energy,explicit,track_id,instrumentalness,key,...,mode,popularity,speechiness,tempo,id_artists,streams_date,streams_year,streams,position,country
0,0.218,2018,0.349,0.511,239836,0.566,1,285pBltuF7vW8TeWk8hdRR,0.000000,6,...,0,89,0.2000,83.903,2,09/12/2019,2019.0,96687.0,155.0,Brazil
1,0.218,2018,0.349,0.511,239836,0.566,1,285pBltuF7vW8TeWk8hdRR,0.000000,6,...,0,89,0.2000,83.903,2,10/12/2019,2019.0,88501.0,183.0,Brazil
2,0.120,2018,0.934,0.351,200186,0.296,0,0u2P5u6lvoDfwTYjAADbn4,0.000000,4,...,0,89,0.0333,115.284,5,24/05/2018,2018.0,52373.0,198.0,Brazil
3,0.120,2018,0.934,0.351,200186,0.296,0,0u2P5u6lvoDfwTYjAADbn4,0.000000,4,...,0,89,0.0333,115.284,5,28/05/2018,2018.0,53482.0,175.0,Brazil
4,0.120,2018,0.934,0.351,200186,0.296,0,0u2P5u6lvoDfwTYjAADbn4,0.000000,4,...,0,89,0.0333,115.284,5,29/05/2018,2018.0,60384.0,152.0,Brazil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14649,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,0,72,0.0664,89.717,6046,30/03/2020,2020.0,117456.0,87.0,Brazil
14650,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,0,72,0.0664,89.717,6046,31/03/2020,2020.0,99292.0,120.0,Brazil
14651,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,0,72,0.0664,89.717,6046,01/04/2020,2020.0,86341.0,145.0,Brazil
14652,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,0,72,0.0664,89.717,6046,02/04/2020,2020.0,85979.0,163.0,Brazil


In [223]:

df_spotify['features_array'] = df_spotify[['valence', 'acousticness', 'danceability','energy' ,'explicit' , \
                                           'instrumentalness', 'key', 'liveness','loudness' , \
                                           'mode','speechiness' ]].values.tolist()
df_spotify

,valence,year,acousticness,danceability,duration_ms,energy,explicit,track_id,instrumentalness,key,...,popularity,speechiness,tempo,id_artists,streams_date,streams_year,streams,position,country,features_array
0,0.218,2018,0.349,0.511,239836,0.566,1,285pBltuF7vW8TeWk8hdRR,0.000000,6,...,89,0.2000,83.903,2,09/12/2019,2019.0,96687.0,155.0,Brazil,"[0.218, 0.349, 0.511, 0.5660000000000001, 1, 0..."
1,0.218,2018,0.349,0.511,239836,0.566,1,285pBltuF7vW8TeWk8hdRR,0.000000,6,...,89,0.2000,83.903,2,10/12/2019,2019.0,88501.0,183.0,Brazil,"[0.218, 0.349, 0.511, 0.5660000000000001, 1, 0..."
2,0.120,2018,0.934,0.351,200186,0.296,0,0u2P5u6lvoDfwTYjAADbn4,0.000000,4,...,89,0.0333,115.284,5,24/05/2018,2018.0,52373.0,198.0,Brazil,"[0.12, 0.934, 0.35100000000000003, 0.296000000..."
3,0.120,2018,0.934,0.351,200186,0.296,0,0u2P5u6lvoDfwTYjAADbn4,0.000000,4,...,89,0.0333,115.284,5,28/05/2018,2018.0,53482.0,175.0,Brazil,"[0.12, 0.934, 0.35100000000000003, 0.296000000..."
4,0.120,2018,0.934,0.351,200186,0.296,0,0u2P5u6lvoDfwTYjAADbn4,0.000000,4,...,89,0.0333,115.284,5,29/05/2018,2018.0,60384.0,152.0,Brazil,"[0.12, 0.934, 0.35100000000000003, 0.296000000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14649,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,72,0.0664,89.717,6046,30/03/2020,2020.0,117456.0,87.0,Brazil,"[0.79, 0.133, 0.47, 0.785, 0, 1.55e-06, 5, 0.0..."
14650,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,72,0.0664,89.717,6046,31/03/2020,2020.0,99292.0,120.0,Brazil,"[0.79, 0.133, 0.47, 0.785, 0, 1.55e-06, 5, 0.0..."
14651,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,72,0.0664,89.717,6046,01/04/2020,2020.0,86341.0,145.0,Brazil,"[0.79, 0.133, 0.47, 0.785, 0, 1.55e-06, 5, 0.0..."
14652,0.790,2020,0.133,0.470,209583,0.785,0,2nMOodYNHBAQ3Kc1QNimZU,0.000002,5,...,72,0.0664,89.717,6046,02/04/2020,2020.0,85979.0,163.0,Brazil,"[0.79, 0.133, 0.47, 0.785, 0, 1.55e-06, 5, 0.0..."
